In [1]:
# Add Matplotlib inline magic command
%matplotlib inline
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import os
import matplotlib as mpl 
from functools import reduce

In [2]:
# Files to load
city_data_to_load = os.path.join("Resources","city_data.csv")
ride_data_to_load = os.path.join("Resources","ride_data.csv")

In [3]:
# Read the city data file and store it in a pandas DataFrame.
city_data_df = pd.read_csv(city_data_to_load)
#city_data_df.head(10)

In [4]:
# Read the ride data file and store it in a pandas DataFrame.
ride_data_df = pd.read_csv(ride_data_to_load)
#ride_data_df.head(10)

In [5]:
# Combine the data into a single dataset
pyber_data_df = pd.merge(ride_data_df, city_data_df, how="left", on=["city", "city"])

# Display the DataFrame
#pyber_data_df.head()

In [6]:
#create data frames for the summaries
total_rides = pyber_data_df.groupby(pyber_data_df["type"]).count()["ride_id"].rename("Total Rides")
total_drivers = city_data_df.groupby(city_data_df["type"]).sum()["driver_count"].rename("Total Drivers")
total_fares = pyber_data_df.groupby(pyber_data_df["type"]).sum()["fare"].rename("Total Fares")
avg_fare_per_ride = pyber_data_df.groupby(pyber_data_df["type"]).mean()["fare"].rename("Average Fare per Ride")
avg_fare_per_driver = (total_fares/total_drivers).rename("Average Fare per Driver")

#total_drivers

### Final Summary Dataframe

In [7]:
#concatenate the series into a single DF
dfs_to_concat = [total_rides,total_drivers,total_fares,avg_fare_per_ride, avg_fare_per_driver]
final_summary_df = pd.concat(dfs_to_concat, axis=1)
#remove index name
final_summary_df.index.name = None
#format df for display
final_summary_df.style.format({"Total Rides": "{:,}", 
                               "Total Drivers": "{:,}", 
                               "Total Fares": "${:,.2f}",
                               "Average Fare per Ride":"${:,.2f}",
                               "Average Fare per Driver":"${:,.2f}"})


,Total Rides,Total Drivers,Total Fares,Average Fare per Ride,Average Fare per Driver
Rural,125,78,"$4,327.93",$34.62,$55.49
Suburban,625,490,"$19,356.33",$30.97,$39.50
Urban,"1,625","2,405","$39,854.38",$24.53,$16.57


In [8]:
#rename pyber_data_df column names
rename_columns = {'city': 'City', 'date':'Date','fare':'Fare', 'ride_id': 'Ride Id','driver_count': 'No. Drivers', 'type':'City Type'}
pyber_data_df.rename(columns = rename_columns, inplace = True)

#set the index to date
pyber_data_df.set_index("Date", inplace=True, drop=False)

pyber_data_df


,City,Date,Fare,Ride Id,No. Drivers,City Type
Date,,,,,,
2019-01-14 10:14:22,Lake Jonathanshire,2019-01-14 10:14:22,13.83,5739410935873,5,Urban
2019-03-04 18:24:09,South Michelleport,2019-03-04 18:24:09,30.24,2343912425577,72,Urban
2019-02-24 04:29:00,Port Samanthamouth,2019-02-24 04:29:00,33.44,2005065760003,57,Urban
2019-02-10 23:22:03,Rodneyfort,2019-02-10 23:22:03,23.44,5149245426178,34,Urban
2019-03-06 04:28:35,South Jack,2019-03-06 04:28:35,34.58,3908451377344,46,Urban
...,...,...,...,...,...,...
2019-04-29 17:04:39,Michaelberg,2019-04-29 17:04:39,13.38,8550365057598,6,Rural
2019-01-30 00:05:47,Lake Latoyabury,2019-01-30 00:05:47,20.76,9018727594352,2,Rural
2019-02-10 21:03:50,North Jaime,2019-02-10 21:03:50,11.11,2781339863778,1,Rural


In [16]:
#Create a new DataFrame for fares and include only the Date, City Type, and Fare columns using the copy() method on the merged DataFrame.
fares_data_df = pyber_data_df[['Date','City Type','Fare']].copy()

#set index, drop extra date
fares_data_df.set_index("Date", inplace=True, drop=True)

#set index data type to datetime
fares_data_df.index = pd.to_datetime(fares_data_df.index)

#check the df with info()
fares_data_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2375 entries, 2019-01-14 10:14:22 to 2019-04-25 10:20:13
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   City Type  2375 non-null   object 
 1   Fare       2375 non-null   float64
dtypes: float64(1), object(1)
memory usage: 55.7+ KB


In [17]:
#Calculate the sum() of fares by the type of city and date using groupby() to create a new DataFrame.
fares_by_type_df = fares_data_df.groupby('City Type').sum()["Fare"]
fares_by_type_df

City Type
Rural        4327.93
Suburban    19356.33
Urban       39854.38
Name: Fare, dtype: float64